In [109]:
from openhunt.logparser import winlogbeat
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [110]:
win = winlogbeat()
spark = SparkSession.builder.appName("Process").config("spark.sql.caseSensitive", "True").getOrCreate()

In [111]:
process_injection = win.extract_nested_fields("dll_injection_2020-01-12160439.json",spark)

[+] Processing a Spark DataFrame..
[+] Reading Mordor file..
[+] Processing Data from Winlogbeat version 7..
[+] DataFrame Returned !


In [112]:
process_injection.createOrReplaceTempView("process_injection")

In [113]:
DLL_Injection = spark.sql(
'''
SELECT 
    b.computer_name,
    b.SourceImage,
    ltrim('0x', b.GrantedAccess) as granted_access,
    b.TargetImage,
    b.SourceProcessId,
    b.SourceProcessGUID,
    c.NewThreadId,
    b.TargetProcessId,
    b.TargetImage,
    c.StartFunction
FROM process_injection c
JOIN process_injection b
    ON c.SourceProcessGuid = b.SourceProcessGUID
    AND b.channel = "Microsoft-Windows-Sysmon/Operational"
    AND b.event_id = 10
WHERE c.channel = "Microsoft-Windows-Sysmon/Operational"
    AND c.event_id = 8
    AND c.StartFunction = "LoadLibraryA" OR c.StartFunction = "LoadLibraryW"
'''
)


In [114]:
df = DLL_Injection.select("computer_name", conv(DLL_Injection.granted_access, 16, 10).alias("granted_access"), "SourceImage", "SourceProcessGUID", "NewThreadId", "TargetProcessId", "TargetImage")

In [115]:
df.createOrReplaceTempView("df")

In [116]:
eid_8_10_minimal_privileges = spark.sql(
'''
SELECT * FROM df WHERE ((INT(granted_access) & 40) == 40) -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
'''
).show(1,False)

+------------------+--------------+-------------------------------------------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|computer_name     |granted_access|SourceImage                                                                                |SourceProcessGUID                     |NewThreadId|TargetProcessId|TargetImage                    |
+------------------+--------------+-------------------------------------------------------------------------------------------+--------------------------------------+-----------+---------------+-------------------------------+
|win10.marvel.local|2097151       |C:\Users\thor\Downloads\injection-master\injection-master\InjectDLL\x64\Debug\InjectDLL.exe|{9d556bd4-b423-5e1b-0000-00104e060d01}|3004       |8288           |C:\WINDOWS\system32\notepad.exe|
+------------------+--------------+---------------------------------------------------------